In [39]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [48]:
EVdf0 = pd.read_csv("EVChargingData2010_2020.csv", low_memory=False)

In [49]:
pd.set_option('display.max_column', None)
EVdf0.head()

,Station Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date (Pacific Time),Total Duration (hh:mm:ss),Charging Time (hh:mm:ss),Energy (kWh),GHG Savings (kg),Gasoline Savings (gallons),Port Type,Port Number,Plug Type,Postal Code,Latitude,Longitude,Ended By,Plug In Event Id,Driver Postal Code,User ID
0,HAMILTON 1,7/29/11 20:17,PDT,7/29/11 23:20,PDT,7/29/11 23:20,3:03:32,1:54:03,6.249457,2.625,0.784,Level 2,2,J1772,94301,37.444572,-122.160309,Plug Out at Vehicle,3,95124.0,3284
1,HAMILTON 1,7/30/11 0:00,PDT,7/30/11 0:02,PDT,7/30/11 0:02,0:02:06,0:01:54,0.106588,0.045,0.013,Level 2,2,J1772,94301,37.444572,-122.160309,Customer,4,94301.0,4169
2,HAMILTON 1,7/30/11 8:16,PDT,7/30/11 12:34,PDT,7/30/11 12:34,4:17:32,4:17:28,14.951777,6.280,1.876,Level 2,2,J1772,94301,37.444572,-122.160309,Plug Out at Vehicle,5,94301.0,4169
3,HAMILTON 1,7/30/11 14:51,PDT,7/30/11 16:55,PDT,7/30/11 16:55,2:03:24,2:02:58,7.159643,3.007,0.899,Level 2,2,J1772,94301,37.444572,-122.160309,Customer,6,94302.0,2545
4,HAMILTON 1,7/30/11 18:51,PDT,7/30/11 20:03,PDT,7/30/11 20:03,1:11:24,0:43:54,1.957765,0.822,0.246,Level 2,2,J1772,94301,37.444572,-122.160309,Plug Out at Vehicle,7,94043.0,3765


In [50]:
EVdf0.dropna(inplace = True)

In [51]:
EVdf0.drop_duplicates(inplace = True)

#### Aggregation

In [90]:
def convert_duration_to_time_window(duration_str, time_window='1H'):
    """
        Convert charging duration from hh:mm:ss format to the specified time window unit.
        
        Parameters:
            duration_str (str): The charging duration in hh:mm:ss format.
            time_window (str): The time window for conversion. Can be '1H', '1D', '1W', or '1M'.
            
        Returns:
            float: The duration converted to the specified time window unit.
        """
    # Convert hh:mm:ss to total hours
    total_hours = pd.Timedelta(duration_str).total_seconds() / 3600

    # Convert based on the specified time window
    if time_window == '1H':
        return total_hours
    elif time_window == '1D':
        return total_hours / 24
    elif time_window == '1W':
        return total_hours / (24 * 7)
    elif time_window == '1M':
        return total_hours / (24 * 30)
    else:
        raise ValueError("Unsupported time window. Choose from '1H', '1D', '1W', '1M'.")

In [91]:
def aggregate_ev_charging(data, time_window='1H'):
    """
    Aggregate energy consumption, total charging duration, and count EV charging events
    within specified time windows.
    
    Parameters:
        dataframe (pd.DataFrame): The EV charging data.
        time_window (str): The time window for aggregation. Can be '1H', '1D', '1W', or '1M'.
        
    Returns:
        pd.DataFrame: Aggregated data with total energy, total charging duration, and count of events.
    """
    dataframe = data.copy()

    dataframe['Start Date'] = pd.to_datetime(dataframe['Start Date'], errors='coerce')

    dataframe = dataframe.dropna(subset=['Start Date'])

    # Ensure the Transaction Date is in datetime format
    dataframe['Start Date'] = pd.to_datetime(dataframe['Start Date'], format="%m/%d/%y %H:%M")

    dataframe.set_index('Start Date', inplace=True)
    
    # Convert Charging Time to hours
    dataframe[f'Charging Time ({time_window})'] = dataframe['Charging Time (hh:mm:ss)'].apply(convert_duration_to_time_window)
    
    # Resample and aggregate data
    aggregated_data = dataframe.groupby('Station Name').resample(time_window).agg({
        'Energy (kWh)': 'sum',
        f'Charging Time ({time_window})': 'sum',
        'User ID': 'count'  # Count of events
    })
    
    # Rename the columns
    aggregated_data.rename(columns={
        'Energy (kWh)': 'Total Energy (kWh)',
        f'Charging Time ({time_window})': f'Total Charging Duration ({time_window})',
        'User ID': 'Event Count'
    }, inplace=True)
    
    return aggregated_data.reset_index()

In [92]:
# Example usage
df_aggregated_hourly = aggregate_ev_charging(EVdf0, '1H')
df_aggregated_daily = aggregate_ev_charging(EVdf0, '1D')
df_aggregated_weekly = aggregate_ev_charging(EVdf0, '1W')
df_aggregated_monthly = aggregate_ev_charging(EVdf0, '1M')

In [93]:
df_aggregated_hourly

,Station Name,Start Date,Total Energy (kWh),Total Charging Duration (1H),Event Count
0,BRYANT 1,2011-10-13 12:00:00,1.630508,0.446944,1
1,BRYANT 1,2011-10-13 13:00:00,0.000000,0.000000,0
2,BRYANT 1,2011-10-13 14:00:00,0.000000,0.000000,0
3,BRYANT 1,2011-10-13 15:00:00,0.000000,0.000000,0
4,BRYANT 1,2011-10-13 16:00:00,0.000000,0.000000,0
...,...,...,...,...,...
1532582,WEBSTER 3,2020-12-31 10:00:00,0.000000,0.000000,0
1532583,WEBSTER 3,2020-12-31 11:00:00,0.000000,0.000000,0
1532584,WEBSTER 3,2020-12-31 12:00:00,0.000000,0.000000,0
1532585,WEBSTER 3,2020-12-31 13:00:00,0.000000,0.000000,0


In [94]:
df_aggregated_daily

,Station Name,Start Date,Total Energy (kWh),Total Charging Duration (1D),Event Count
0,BRYANT 1,2011-10-13,8.262052,2.226944,2
1,BRYANT 1,2011-10-14,6.259466,1.698333,1
2,BRYANT 1,2011-10-15,4.622894,1.595278,1
3,BRYANT 1,2011-10-16,0.000000,0.000000,0
4,BRYANT 1,2011-10-17,4.023361,1.510556,1
...,...,...,...,...,...
63897,WEBSTER 3,2020-12-27,0.000000,0.000000,0
63898,WEBSTER 3,2020-12-28,0.000000,0.000000,0
63899,WEBSTER 3,2020-12-29,37.291000,6.106667,3
63900,WEBSTER 3,2020-12-30,47.706000,7.938333,2


In [95]:
df_aggregated_weekly

,Station Name,Start Date,Total Energy (kWh),Total Charging Duration (1W),Event Count
0,BRYANT 1,2011-10-16,19.144412,5.520556,4
1,BRYANT 1,2011-10-23,59.982234,16.720833,8
2,BRYANT 1,2011-10-30,75.748505,19.700833,9
3,BRYANT 1,2011-11-06,88.177382,27.249722,12
4,BRYANT 1,2011-11-13,76.546939,23.666667,13
...,...,...,...,...,...
9163,WEBSTER 3,2020-12-06,102.762000,19.007500,6
9164,WEBSTER 3,2020-12-13,100.447000,18.252500,5
9165,WEBSTER 3,2020-12-20,73.224000,13.582222,9
9166,WEBSTER 3,2020-12-27,87.159000,14.263611,6


In [96]:
df_aggregated_monthly

,Station Name,Start Date,Total Energy (kWh),Total Charging Duration (1M),Event Count
0,BRYANT 1,2011-10-31,166.375035,44.188333,23
1,BRYANT 1,2011-11-30,285.949489,86.807778,42
2,BRYANT 1,2011-12-31,275.542869,82.283611,45
3,BRYANT 1,2012-01-31,305.337793,99.055000,50
4,BRYANT 1,2012-02-29,355.545309,107.422222,58
...,...,...,...,...,...
2128,WEBSTER 3,2020-08-31,327.143000,67.417500,31
2129,WEBSTER 3,2020-09-30,328.939000,57.949444,22
2130,WEBSTER 3,2020-10-31,558.985000,106.877778,34
2131,WEBSTER 3,2020-11-30,489.256000,82.973889,33
